In [29]:
import pandas as pd
import numpy as np
import torch

In [26]:
df_train_clip = pd.read_csv("/home/docker_current/datasets/train.csv")

with open('image_feat_array.npy', 'rb') as f:
    image_feat_array = np.load(f)

with open('text_feat_array.npy', 'rb') as f:
    text_feat_array = np.load(f)

df_train_clip['image_features'] = [i.reshape(1,-1) for i in image_feat_array]
df_train_clip['text_features'] = [i.reshape(1,-1) for i in text_feat_array]
df_train_clip.head(3)

,id,description,object_img,image_features,text_features
0,520,Фотография. Г. Пермь. Здание горисполкома. ПО...,799,"[[1.454, 1.191, 0.521, 1.783, -0.4033, -0.615,...","[[-0.24118489, 0.25865138, -0.14606075, 1.2203..."
1,1817,Фотонегатив пленочный. Труппа театра «У моста»...,854,"[[0.7256, 0.1669, 0.1838, 1.635, -0.2441, 0.46...","[[-0.12055853, 0.15714812, 0.032904923, 0.7815..."
2,188,"Изразец гладкий расписной ""пермский""- карниз А...",1794,"[[0.3489, -0.03116, 0.238, 1.218, 0.1726, 0.74...","[[0.30912644, 0.8710553, -0.00015427265, -0.00..."


In [30]:
cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)

def get_similarity(df, index_text, index_image):
    sim =cos(torch.Tensor(df_train_clip['text_features'][index_text]), 
                        torch.Tensor(df_train_clip['image_features'][index_image]))
    return sim

In [81]:
%%time

count_true_map = 0
preds = []

for ind_text in range(len(df_train_clip)):

    sims = []

    for ind_image in range(len(df_train_clip)):
        sim = get_similarity(df_train_clip, ind_text, ind_image)
        sims.append(sim)

    sims = np.array(sims)
    ind_max = np.argmax(sims)
    preds.append(ind_max)
    # print(ind_max, sims[ind_max])

    if ind_max == ind_text:
        count_true_map += 1

count_true_map

CPU times: user 4min 10s, sys: 87.9 ms, total: 4min 10s
Wall time: 4min 11s


382

In [82]:
count_true_map, len(df_train_clip)

(382, 2098)

In [89]:
# !pip install sklearn
from sklearn.metrics import r2_score

y_test = df_train_clip['object_img']

print("R2 score:", r2_score(list(y_test), preds))

R2 score: -0.6248734862466188
